# Exercise 03 : Aggregations

## Create a connection to the database using the library sqlite3


In [25]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("../data/checking-logs.sqlite")

## Get the schema of the table test


In [26]:
schema = pd.read_sql("PRAGMA table_info(test);", conn)
schema

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## Get only the first 10 rows of the table test to check what the table looks like


In [27]:
preview = pd.read_sql("SELECT * FROM test LIMIT 10;", conn)
preview

,uid,labname,first_commit_ts,first_view_ts
0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


## Find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query


In [28]:
pd.read_sql("PRAGMA table_info(deadlines);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [29]:
query_min = """
SELECT test.uid, 
       MIN((julianday(datetime(deadlines.deadlines, 'unixepoch')) - julianday(test.first_commit_ts)) * 24) AS delta_hours
FROM test
JOIN deadlines ON test.labname = deadlines.labs
WHERE test.labname != 'project1'
GROUP BY test.uid
ORDER BY delta_hours ASC
LIMIT 1;
"""

df_min = pd.read_sql(query_min, conn)
df_min

,uid,delta_hours
0,user_25,2.867236


## Do the same thing, but for the maximum, using only one query, the dataframe name is df_max


In [30]:
query_max = """
SELECT test.uid, 
       MAX((julianday(datetime(deadlines.deadlines, 'unixepoch')) - julianday(test.first_commit_ts)) * 24) AS delta_hours
FROM test
JOIN deadlines ON test.labname = deadlines.labs
WHERE test.labname != 'project1'
GROUP BY test.uid
ORDER BY delta_hours DESC
LIMIT 1;
"""
df_max = pd.read_sql(query_max, conn)
df_max

,uid,delta_hours
0,user_30,202.38473


## Do the same thing but for the average, using only one query, this time your dataframe should not include the uid column, and the dataframe name is df_avg


In [31]:
query_avg = """
SELECT 
    AVG((julianday(datetime(deadlines.deadlines, 'unixepoch')) - julianday(test.first_commit_ts)) * 24) AS avg_delta_hours
FROM test
JOIN deadlines ON test.labname = deadlines.labs
WHERE test.labname != 'project1';
"""
df_avg = pd.read_sql(query_avg, conn)
df_avg

,avg_delta_hours
0,89.687686


## We want to test the hypothesis that the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. To do this, you need to calculate the correlation coefficient between the number of pageviews and the difference


In [32]:
query_views_diff = """
SELECT 
    test.uid,
    AVG((julianday(datetime(deadlines.deadlines, 'unixepoch')) - julianday(test.first_commit_ts)) * 24) AS avg_diff,
    COUNT(pageviews.uid) AS pageviews
FROM test
JOIN deadlines ON test.labname = deadlines.labs
LEFT JOIN pageviews ON test.uid = pageviews.uid
WHERE test.labname != 'project1'
GROUP BY test.uid;
"""
views_diff = pd.read_sql(query_views_diff, conn)
views_diff

,uid,avg_diff,pageviews
0,user_1,65.119644,140
1,user_10,75.242310,445
2,user_14,159.568696,429
3,user_17,62.207513,235
4,user_18,6.367907,9
5,user_19,99.440298,64
6,user_21,96.111041,40
7,user_25,93.474751,895
8,user_28,86.793652,745
9,user_3,105.738041,1585


## Use the Pandas method corr() to calculate the correlation coefficient between the number of pageviews and the difference


In [33]:
correlation = views_diff['avg_diff'].corr(views_diff['pageviews'])
correlation

0.18504199436324817

## Close the connection


In [34]:
conn.close()